In [58]:
from functools import reduce
from PySimpleAutomata import DFA, automata_IO
import json
import pdb

In [59]:
with open('fsm/22-Num.json', 'r') as f:
    j = json.load(f)

In [60]:
alphabet = list(reduce(lambda acc, x: acc | set(x.keys()), list(j.values()), set()))

In [61]:
states = list(reduce(lambda acc, x: acc + list(x.values()), list(j.values()), [])+list(j.keys()))

In [62]:
initial_state = '0'
accepting_states = ['2']

In [63]:
trans = []
for s0, transitions in j.items():
    for x, s1 in transitions.items():
        trans.append([s0, 'Ɛ' if x == '' else x, list(s1)[0]])

In [69]:
newNfa = {
    "alphabet": alphabet,
    "states": states,
    "initial_states": [initial_state],
    "accepting_states": accepting_states,
    "transitions": trans
}

In [53]:
lex = {
    "alphabet": ['letra', 'digito', 'especial', 'controle'],
    "states": ['A', 'B', 'C', 'D', 'E'],
    "initial_state": 'A',
    "accepting_states": ['B', 'C', 'D', 'E'],
    "transitions": [
        ['A', 'letra', 'B'],
        ['A', 'digito', 'C'],
        ['A', 'especial', 'D'],
        ['A', 'controle', 'E'],
        ['B', 'letra', 'B'],
        ['B', 'digito', 'B'],
        ['C', 'digito', 'C'],
        ['D', 'especial', 'D'],
        ['E', 'letra', 'E'],
        ['E', 'digito', 'E'],
        ['E', 'especial', 'E']
    ],
},

In [70]:
automata_IO.nfa_json_importer('lex.json')

TypeError: list indices must be integers or slices, not str

In [71]:
with open('lex.json', 'w') as f:
    json.dump(lex, f)
automata_IO.dfa_to_dot(
    automata_IO.dfa_json_importer('lex.json'),
    'lex',
    '.'
)

TypeError: list indices must be integers or slices, not str

In [ ]:
states = {
    'A': {'next': {'letra': 'B','digito': 'C', 'especial': 'D', 'controle': 'E'}, 'final': False},
    'B': {'next': {'letra': 'B','digito': 'B'}, 'final': True, 'type':'letter'},
    'C': {'next': {'digito': 'C'}, 'final': True, 'type':'digit'},
    'D': {'next': {'especial': 'D'}, 'final': True, 'type':'especial'},
    'E': {'next': {'letra': 'E', 'digito': 'E', 'especial': 'E'}, 'final': True, 'type': 'remark'},
}

In [65]:
alphabet

['', '+', ':Int', '.', '-', 'digit', 'E']

In [66]:
with open('nfa_example.json', 'w') as f:
    json.dump(newNfa, f)

In [68]:
automata_IO.nfa_to_dot(
    automata_IO.nfa_json_importer('nfa_example.json'),
    'nfa_example',
    '.'
)

TypeError: unhashable type: 'list'

In [40]:
j['2'] = {}

In [41]:
def e_closure(state, transitions):
    result = {state}
    if '' in list(transitions[state].keys()):
        nextState = transitions[state]['']
        result |= {nextState} | e_closure(nextState, transitions)
    return result

In [91]:
j

{'0': {'': '0-0'},
 '0-2': {'': '1'},
 '0-4': {'': '1'},
 '0-0': {'Int': '0-1', '“.”': '0-3'},
 '0-1': {'': '0-1-0'},
 '0-1-2': {'': '0-2'},
 '0-1-0': {'“.”': '0-1-1', '': '0-1-2'},
 '0-1-1': {'': '0-1-1-0'},
 '0-1-1-0': {'': '0-1-2', 'digit': '0-1-1-1'},
 '0-1-1-1': {'': '0-1-1-0'},
 '0-3': {'Int': '0-4'},
 '1': {'': '1-0'},
 '1-3': {'': '2'},
 '1-0': {'“E”': '1-1', '': '1-3'},
 '1-1': {'': '1-1-0'},
 '1-1-1': {'': '1-2'},
 '1-1-2': {'': '1-2'},
 '1-1-0': {'“+”': '1-1-1', '“-”': '1-1-2', '': '1-1-2'},
 '1-2': {'Int': '1-3'},
 '2': {}}

In [92]:
e_closure('0', j)

{'0', '0-0'}

In [176]:
def nfa2dfa(nfa):
    transitions = []
    searchStates = [{'0'}]
    searchedStates = [{'0'}]
    while searchStates:
        #pdb.set_trace()
        currState = reduce(lambda acc, x: acc | e_closure(x, j), searchStates.pop(0), set())
        trans = {}
        for item in currState:
            for symbol, s1 in j[item].items():
                if symbol != '':
                    if not symbol in trans:
                        trans[symbol] = e_closure(s1, j)
                    else: trans[symbol] |= e_closure(s1, j)
            #pdb.set_trace()
            for symbol, s1 in trans.items():
                if not s1 in (searchStates + searchedStates):
                    searchStates.append(s1)
                transitions.append([';'.join(sorted(currState)), symbol, ';'.join(sorted(s1))])
            searchedStates.append(currState)
    return transitions

In [177]:
z = nfa2dfa(0)

In [178]:
states = reduce(lambda acc, x: acc | {x[0], x[2]}, z, set())

In [179]:
alphabet = reduce(lambda acc, x: acc | {x[1]}, z, set())

In [181]:
states

{'0-1-1-0;0-1-1-1;0-1-2;0-2;1;1-0;1-3;2',
 '0-1-1;0-1-1-0;0-1-2;0-2;1;1-0;1-3;2',
 '0-1;0-1-0;0-1-2;0-2;1;1-0;1-3;2',
 '0-3',
 '0-4;1;1-0;1-3;2',
 '0;0-0',
 '1-1-1;1-2',
 '1-1-2;1-2',
 '1-1;1-1-0;1-1-2;1-2',
 '1-3;2'}

In [182]:
initial_state = '0;0-0'
accepting_states = ['0-1-1-0;0-1-1-1;0-1-2;0-2;1;1-0;1-3;2','0-1-1;0-1-1-0;0-1-2;0-2;1;1-0;1-3;2','0-1;0-1-0;0-1-2;0-2;1;1-0;1-3;2','0-4;1;1-0;1-3;2','1-3;2']

In [185]:
newDfa = {
    "alphabet": list(alphabet),
    "states": list(states),
    "initial_state": initial_state,
    "accepting_states": accepting_states,
    "transitions": z
}

In [186]:
with open('dfa_example.json', 'w') as f:
    json.dump(newDfa, f)

In [188]:
automata_IO.dfa_to_dot(
    automata_IO.dfa_json_importer('dfa_example.json'),
    'dfa_example',
    '.'
)

In [57]:
automata_IO.dfa_json_importer('dfa_example.json'),

FileNotFoundError: [Errno 2] No such file or directory: 'dfa_example.json'